# 3. Automatizando Protocolos Clínicos com `gemx`

Este notebook demonstra como invocar as automações clínicas do `gemini_megapack` diretamente de um ambiente Python. Isso permite integrar os protocolos gerados por IA em outras aplicações, como um prontuário eletrônico, um sistema de suporte à decisão ou um dashboard.

**Objetivo:** Executar a automação do "Bundle de Sepse" e exibir o protocolo gerado.

In [ ]:
import subprocess
import os
from IPython.display import display, Markdown

# O gemx.sh está na pasta k8s/gemini_megapack/
# Vamos assumir que nosso notebook está em /workspaces/Gemini/matheus_apple_med_dev_suite/notebooks
# O CWD (Current Working Directory) do notebook é a raiz do projeto.
GEMX_SCRIPT_PATH = os.path.abspath("k8s/gemini_megapack/gemx.sh")
AUTOMATIONS_DIR = os.path.abspath("k8s/gemini_megapack/automations")

# Verificando se o script existe
if not os.path.exists(GEMX_SCRIPT_PATH):
    print(f"ERRO: gemx.sh não encontrado em {GEMX_SCRIPT_PATH}")
    print("Por favor, ajuste o caminho se necessário.")

### Passo 1: Criar uma Função para Executar a Automação

Vamos encapsular a chamada do `subprocess` em uma função que recebe o nome do arquivo de automação e retorna a saída em Markdown.

In [ ]:
def run_automation(automation_name: str) -> str:
    """Executa uma automação do gemx e retorna a saída."""
    automation_file = os.path.join(AUTOMATIONS_DIR, automation_name)
    
    if not os.path.exists(automation_file):
        return f"## Erro
Arquivo de automação não encontrado: `{automation_file}`"
    
    command = [GEMX_SCRIPT_PATH, "auto", "run", automation_file]
    
    print(f"Executando: `{' '.join(command)}`...")
    try:
        # A execução pode demorar um pouco, dependendo da complexidade do prompt
        result = subprocess.run(
            command,
            capture_output=True, text=True, check=True, timeout=180
        )
        # A saída principal do gemx gen é o stdout
        return result.stdout
    except FileNotFoundError:
        return f"## Erro
O script `{GEMX_SCRIPT_PATH}` não foi encontrado ou não é executável."
    except subprocess.CalledProcessError as e:
        return f"## Erro na Execução
**Exit Code:** `{e.returncode}`
**Stderr:**
```
{e.stderr}
```"
    except subprocess.TimeoutExpired:
        return "## Erro
O comando excedeu o tempo limite de 180 segundos."

### Passo 2: Executar o Protocolo de Sepse

Agora, vamos chamar nossa função para gerar o bundle de sepse. A biblioteca `IPython.display.Markdown` renderizará a saída formatada diretamente na célula.

In [ ]:
# O nome do arquivo da automação que queremos rodar
automation_to_run = "sepse_bundle.yaml"

markdown_output = run_automation(automation_to_run)

# Exibir o resultado formatado
display(Markdown(markdown_output))

### Passo 3: Executar Outro Protocolo (HDA)

Para mostrar a flexibilidade, podemos facilmente executar um protocolo diferente, como o de Hemorragia Digestiva Alta (HDA).

In [ ]:
hda_output = run_automation("hda_triage.yaml")
display(Markdown(hda_output))

## Conclusão e Próximos Passos

A capacidade de invocar programaticamente as automações do `gemx` abre um leque de possibilidades:

- **Integração com Prontuários:** Um sistema poderia pré-preencher um protocolo com dados do paciente antes de chamar a automação.
- **Aplicações Web Interativas:** Uma aplicação web (como a que existe em `k8s/gemini_megapack/web/`) pode oferecer uma interface gráfica para selecionar e executar esses protocolos.
- **Análise de Resultados:** A saída em Markdown pode ser parseada para extrair informações estruturadas, como doses de medicamentos ou checklists, para validação ou logging.

Esta série de notebooks cobriu o ciclo desde a busca de dados brutos (`medcli`), passando pela sua visualização (`pandas`), até a geração de protocolos complexos e clinicamente relevantes com o `gemx`.